In [1]:
!pip install transformers datasets rouge_score torch accelerate --quiet

  Preparing metadata (setup.py) ... done


In [2]:
import torch
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
from rouge_score import rouge_scorer
import pandas as pd
import numpy as np

# Loading the dataset
print("Loading CNN/DailyMail dataset...")
dataset = load_dataset("cnn_dailymail", "3.0.0")

train_dataset = dataset["train"].select(range(1000))
val_dataset = dataset["validation"].select(range(100))
test_dataset = dataset["test"].select(range(100))

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

Loading CNN/DailyMail dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Train size: 1000
Validation size: 100
Test size: 100


In [3]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

print(f"Loaded model: {model_name}")

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Loaded model: facebook/bart-large-cnn


In [4]:
def preprocess_function(examples):
    """Preprocess the dataset for BART fine-tuning"""
    # Tokenize inputs (articles)
    inputs = tokenizer(
        examples["article"],
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )

    # Tokenize targets (summaries)
    targets = tokenizer(
        examples["highlights"],
        max_length=128,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )

    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply preprocessing
print("Preprocessing datasets...")
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Preprocessing datasets...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
# Custom compute metrics function for ROUGE
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for pred, label in zip(decoded_preds, decoded_labels):
        scores = scorer.score(label, pred)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    return {
        'rouge1': np.mean(rouge1_scores),
        'rouge2': np.mean(rouge2_scores),
        'rougeL': np.mean(rougeL_scores)
    }

In [7]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    return_tensors="pt"
)


# Training arguments
training_args = TrainingArguments(
    output_dir="./bart-cnn-summarization",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    gradient_accumulation_steps=2,
    fp16=True,
    report_to="none"
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3479462436.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
trainer.train()

Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=375, training_loss=1.4560780131022135, metrics={'train_runtime': 516.1921, 'train_samples_per_second': 5.812, 'train_steps_per_second': 0.726, 'total_flos': 6501313806336000.0, 'train_loss': 1.4560780131022135, 'epoch': 3.0})

In [9]:
def generate_summary(text, max_length=128):
    """Generate summary for a given text"""
    inputs = tokenizer.encode(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True
    )

    # Move the input tensor to the same device as the model
    device = model.device
    inputs = inputs.to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            inputs,
            max_length=max_length,
            min_length=30,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Generate summaries for test set
print("Generating summaries for test set...")
generated_summaries = []
reference_summaries = []

for i, example in enumerate(test_dataset):
    if i >= 10:
        break

    article = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
    reference = tokenizer.decode(example['labels'], skip_special_tokens=True)

    generated = generate_summary(article)

    generated_summaries.append(generated)
    reference_summaries.append(reference)

    print(f"\n--- Example {i+1} ---")
    print(f"Generated: {generated}")
    print(f"Reference: {reference}")

Generating summaries for test set...

--- Example 1 ---
Generated: Palestinian Authority officially becomes 123rd member of International Criminal Court .
Palestinian foreign minister: Today brings us closer to our shared goals of justice and peace .
Israel and the U.S. opposed the Palestinians' efforts to join the court .
Reference: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

--- Example 2 ---
Generated: Stray dog was hit by a car, buried and found four days later .
Theia suffered a dislocated jaw, leg injuries and a caved-in sinus cavity .
A fundraising page has raised more than $10,000 for Theia .
Reference: Theia, a bully breed mix, was apparently hit by a car, whacked with a hammer and buried in a field .
"She's a true miracle dog and she deserves a good life," says Sara Mellado, who is lookin

In [10]:
def evaluate_rouge_scores(generated_summaries, reference_summaries):
    """Compute detailed ROUGE scores"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge1_f = []
    rouge1_p = []
    rouge1_r = []
    rouge2_f = []
    rouge2_p = []
    rouge2_r = []
    rougeL_f = []
    rougeL_p = []
    rougeL_r = []

    for gen, ref in zip(generated_summaries, reference_summaries):
        scores = scorer.score(ref, gen)

        rouge1_f.append(scores['rouge1'].fmeasure)
        rouge1_p.append(scores['rouge1'].precision)
        rouge1_r.append(scores['rouge1'].recall)

        rouge2_f.append(scores['rouge2'].fmeasure)
        rouge2_p.append(scores['rouge2'].precision)
        rouge2_r.append(scores['rouge2'].recall)

        rougeL_f.append(scores['rougeL'].fmeasure)
        rougeL_p.append(scores['rougeL'].precision)
        rougeL_r.append(scores['rougeL'].recall)

    results = {
        'ROUGE-1': {
            'F1': np.mean(rouge1_f),
            'Precision': np.mean(rouge1_p),
            'Recall': np.mean(rouge1_r)
        },
        'ROUGE-2': {
            'F1': np.mean(rouge2_f),
            'Precision': np.mean(rouge2_p),
            'Recall': np.mean(rouge2_r)
        },
        'ROUGE-L': {
            'F1': np.mean(rougeL_f),
            'Precision': np.mean(rougeL_p),
            'Recall': np.mean(rougeL_r)
        }
    }

    return results

# Evaluate and create results table
print("\nEvaluating ROUGE scores...")
rouge_results = evaluate_rouge_scores(generated_summaries, reference_summaries)

# Create and display ROUGE score table
rouge_df = pd.DataFrame(rouge_results).round(4)
print("\n=== ROUGE SCORE TABLE ===")
print(rouge_df)

# Save results
rouge_df.to_csv("rouge_scores.csv")
print("\nROUGE scores saved to 'rouge_scores.csv'")


Evaluating ROUGE scores...

=== ROUGE SCORE TABLE ===
           ROUGE-1  ROUGE-2  ROUGE-L
F1          0.3974   0.1735   0.2968
Precision   0.3904   0.1636   0.2876
Recall      0.4270   0.1965   0.3230

ROUGE scores saved to 'rouge_scores.csv'


In [11]:
results_summary = {
    'Metric': [],
    'Score': [],
    'Standard Deviation': []
}

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
all_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

for gen, ref in zip(generated_summaries, reference_summaries):
    scores = scorer.score(ref, gen)
    all_scores['rouge1'].append(scores['rouge1'].fmeasure)
    all_scores['rouge2'].append(scores['rouge2'].fmeasure)
    all_scores['rougeL'].append(scores['rougeL'].fmeasure)

for metric, scores in all_scores.items():
    results_summary['Metric'].append(f"{metric.upper()}-F1")
    results_summary['Score'].append(f"{np.mean(scores):.4f}")
    results_summary['Standard Deviation'].append(f"{np.std(scores):.4f}")

summary_df = pd.DataFrame(results_summary)
print("\n=== DETAILED RESULTS SUMMARY ===")
print(summary_df)


=== DETAILED RESULTS SUMMARY ===
      Metric   Score Standard Deviation
0  ROUGE1-F1  0.3974             0.1020
1  ROUGE2-F1  0.1735             0.1067
2  ROUGEL-F1  0.2968             0.0921
